In [1]:
!pip3 install cv2
!pip3 install numpy
!pip3 install cvzone
!pip3 install mediapipe
!pip3 install protobuf==3.20.1

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Created wheel for cvzone: filename=cvzone-1.5.6-py3-none-any.whl size=18766 sha256=d0061389b17722dac909bc0de716291d6ea267e7db2a7a2346ff6e3c4086c5b2
  Stored in directory: c:\users\micke\appdata\local\pip\cache\wheels\c1\e8\e9\80f482161ba9f5dcf4832b76ac70540edd11a3136a58445c52
Successfully built cvzone
     ---------------------------------------- 48.6/48.6 MB 8.1 MB/s eta 0:00:00


     -------------------------------------- 905.1/905.1 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.15.4
    Uninstalling protobuf-3.15.4:
      Successfully uninstalled protobuf-3.15.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 1.2.10 requires tensorboard!=2.5.0,>=2.2.0, but you have tensorboard 1.15.0 which is incompatible.
pytorch-lightning 1.2.10 requires tqdm>=4.41.0, but you have tqdm 4.28.1 which is incompatible.
isr 2.2.0 requires tensorflow==2.0.0, but you have tensorflow 1.15.5 which is incompatible.


In [2]:
import cv2
import numpy
from cvzone.SelfiSegmentationModule import SelfiSegmentation

In [12]:
cam = cv2.VideoCapture("vid.mp4")
vw  = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
vh = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [13]:
body_outline_source = cv2.imread("body_outline.png",cv2.IMREAD_GRAYSCALE)
body_outline_ratio = body_outline_source.shape[1]/body_outline_source.shape[0]
body_outline_source = cv2.resize(
    body_outline_source,
    (int(body_outline_ratio*vh),vh),
    cv2.INTER_AREA
)

In [14]:
video_width_center = vw//2
body_width_center =  body_outline_source.shape[1]//2

In [15]:
body_outline_mask = numpy.ones((vh,vw,3),dtype=numpy.uint8)
for y in range(0,body_outline_source.shape[0],1):
    for x in range(0,body_outline_source.shape[1],1):
        if(body_outline_source[y,x]!=255):
            body_outline_mask[y,video_width_center + (x - body_width_center),0] = 0
            body_outline_mask[y,video_width_center + (x - body_width_center),1] = 0
            body_outline_mask[y,video_width_center + (x - body_width_center),2] = 0
body_outline_mask = cv2.resize(body_outline_mask,(512,512),cv2.INTER_NEAREST)
body_outline_indicator = (1-body_outline_mask) * 255

In [16]:
segmentor = SelfiSegmentation()

In [17]:
cv2.namedWindow("test")
img_counter = 0
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break

    frame = cv2.resize(frame,(512,512), cv2.INTER_AREA)
    frame = numpy.multiply(frame,body_outline_mask) + body_outline_indicator

    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, segmentor.removeBG(frame, (255,255,255), threshold=0.99))
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()
cv2.destroyAllWindows()

opencv_frame_0.png written!
opencv_frame_1.png written!
opencv_frame_2.png written!
opencv_frame_3.png written!
opencv_frame_4.png written!
opencv_frame_5.png written!
Escape hit, closing...
